In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
os.listdir()

['asian.jpg',
 'FaceReg.ipynb',
 'facereg.keras',
 'facereg2.keras',
 'girl.jpg',
 'india.jpg']

In [3]:
# Load model
model_path = 'facereg2.keras'
model = load_model(model_path)
 
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 63, 63,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 63, 63,    │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 63, 63,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 61, 61,    │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 61, 61,    │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 61, 61,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 61, 61,    │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 61, 61,    │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 61, 61,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 30, 30,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 30, 30,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 30, 30,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 30, 30,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 28, 28,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 28, 28,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 28, 28,    │          0 │ batch_normalizat

 Total params: 22,636,343 (86.35 MB)

 Trainable params: 6,599 (25.78 KB)

 Non-trainable params: 22,616,544 (86.28 MB)

 Optimizer params: 13,200 (51.57 KB)

In [4]:
print(model.input_shape)

(None, 128, 128, 3)


In [5]:
def preprocess_image(img_path):
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Không thể đọc ảnh từ {img_path}")
    
    # Chuyển sang grayscale để phát hiện khuôn mặt
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Sử dụng Haar cascade để phát hiện khuôn mặt
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)
    
    if len(faces) == 0:
        # Thử cascade khác nếu không phát hiện được
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')
        faces = face_cascade.detectMultiScale(gray, 1.1, 5)
    
    if len(faces) == 0:
        print("Cảnh báo: Không phát hiện khuôn mặt rõ ràng, sẽ xử lý toàn bộ ảnh")
        face = cv2.resize(img, (128, 128))  # Thay đổi từ 224x224 thành 128x128
        face = face.astype('float32') / 255.0
        return np.expand_dims(face, axis=0), (0, 0, img.shape[1], img.shape[0])
    
    # Lấy khuôn mặt đầu tiên
    (x, y, w, h) = faces[0]
    face = img[y:y+h, x:x+w]
    
    # Resize về 128x128
    face = cv2.resize(face, (128, 128))
    
    # Chuẩn hóa pixel values
    face = face.astype('float32') / 255.0
    
    # Thêm batch dimension
    face = np.expand_dims(face, axis=0)
    
    return face, (x, y, w, h)

In [6]:
def predict_demographics(img_path):
    # Tiền xử lý ảnh
    face, (x, y, w, h) = preprocess_image(img_path)
    
    predictions = model.predict(face)
    
    # Giả sử model trả về 3 outputs: age, gender, race
    age = predictions[0][0] * 100  # Giả sử tuổi được chuẩn hóa về [0,1]
    gender_prob = predictions[1][0]
    race_prob = predictions[2][0]
    
    # Xác định giới tính
    gender = "Nữ" if gender_prob[0] > 0.5 else "Nam"
    gender_confidence = max(gender_prob[0], 1 - gender_prob[0])
    
    # Xác định sắc tộc
    race_classes = ['White', 'Black', 'Asian', 'Indian', 'Others']
    race_idx = np.argmax(race_prob)
    race = race_classes[race_idx]
    race_confidence = race_prob[race_idx]
    
    return {
        'age': int(age),
        'gender': gender,
        'gender_confidence': float(gender_confidence),
        'race': race,
        'race_confidence': float(race_confidence),
        'face_coordinates': (int(x), int(y), int(w), int(h))
    }

In [7]:
def display_results(img_path, results):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    x, y, w, h = results['face_coordinates']
    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    info = f"Age: {results['age']}\nGender: {results['gender']} ({results['gender_confidence']:.2f})\nRace: {results['race']} ({results['race_confidence']:.2f})"
    
    y_text = y - 10 if y - 10 > 10 else y + h + 20
    for i, line in enumerate(info.split('\n')):
        cv2.putText(img, line, (x, y_text + i*20), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
    
    # Hiển thị ảnh
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Kết quả nhận diện:
Tuổi: 32
Giới tính: Nam (độ tin cậy: 0.95)
Sắc tộc: White (độ tin cậy: 0.48)


C:\Users\vuong\AppData\Local\Temp\ipykernel_9096\4117423036.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  'age': int(age),


In [10]:
image_path = 'india.jpg'

# Dự đoán
results = predict_demographics(image_path)

# Hiển thị kết quả
print("Kết quả nhận diện:")
print(f"Tuổi: {results['age']}")
print(f"Giới tính: {results['gender']} (độ tin cậy: {results['gender_confidence']:.2f})")
print(f"Sắc tộc: {results['race']} (độ tin cậy: {results['race_confidence']:.2f})")

# Hiển thị ảnh với kết quả
display_results(image_path, results)